In [1]:
import numpy as np
import strawberryfields as sf
import random
from typing import List

In [2]:
batch_size = 100
alphas = list(np.arange(0.0, 2.1, 0.05))
betas = list(np.arange(-8.0, 2.0, 0.1))
A = 1
A = -1

In [3]:
def create_codeword(letters: List[float], codeword_size=10) -> List[float]:
        return [random.choice(letters) for _ in range(codeword_size)]

In [4]:
codeword = create_codeword([A*alphas[15], -A*alphas[15]], codeword_size=batch_size)

In [5]:
p_1 = codeword.count(A*alphas[15])/len(codeword)
p_0 = 1 - p_1

In [6]:
len(betas)


100

In [7]:
len(codeword)

100

In [8]:
eng = sf.Engine(backend="fock", backend_options={
        "cutoff_dim": 7,
        "batch_size": len(codeword),
    })

In [9]:
circuit = sf.Program(1)

beta = circuit.params("beta")
alpha = circuit.params("alpha")

with circuit.context as q:
    sf.ops.Dgate(alpha, 0.0) | q[0]
    sf.ops.Dgate(beta, 0.0) | q[0]

In [11]:
results = eng.run(circuit, args={
            "beta": betas[0],
            "alpha": codeword[0]
        })
        
# get the probability of |0>
p_zero = results.state.fock_prob([0])

# get the porbability of anything by |0>
p_one = 1 - p_zero